In [33]:
import os
import pandas as pd

from tqdm.auto import tqdm
from haystack_integrations.document_stores.elasticsearch import ElasticsearchDocumentStore
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack_integrations.components.retrievers.elasticsearch import ElasticsearchBM25Retriever
from haystack import Document
from haystack import Pipeline

In [34]:
dataset_path = 'datasets/allegro/'

embedder_model = "sentence-transformers/multi-qa-mpnet-base-dot-v1"

## Lauch ElasticSearch in Docker

In [35]:
# Run docker-compose up while in main directory in order to launch elasticsearch instance.

In [36]:
document_store = ElasticsearchDocumentStore(hosts='http://localhost:9200')
document_store.delete_documents(document_ids=[document.id for document in document_store.filter_documents(filters={})])

## Index Passages

In [39]:
passages = pd.read_json(os.path.join(dataset_path, 'passages.jl'), lines=True, chunksize=int(1e6))

for batch in tqdm(passages):
    if 'title' in batch:
        batch['title'] = batch['title'].fillna('')
        batch['text'] = batch.apply(lambda r: r['title'] + ' ' + r['text'], axis=1)
    
    batch = batch.rename(columns={'id': 'passage-id', 'text': 'content'})
    batch_as_dicts = batch.to_dict(orient='records')
    document_store.write_documents([Document(content=passageDict['content'], id=passageDict['passage-id']) for passageDict in batch_as_dicts])

0it [00:00, ?it/s]

In [40]:
document_store.count_documents()

921

## Retrieve Similar Passages

In [41]:
# Use a retriever that utilizes BM25 algorithm (bag-of-words based)
retriever = ElasticsearchBM25Retriever(document_store=document_store)

pipe = Pipeline()
pipe.add_component("retriever", retriever)

In [42]:
questions = pd.read_json(os.path.join(dataset_path, 'questions-test.jl'), lines=True)
questions.shape

(900, 2)

In [44]:
preds = []

for _, row in tqdm(questions.iterrows()):
    top_passages = pipe.run({"retriever": {"query": row['text']}})
    
    print(top_passages)
    
    for passage in top_passages['retriever']['documents']:
        passage = passage.to_dict()
        preds.append({
            'question-id': row['id'],
            'passage-id': passage['id'],
            'score': passage['score'],
        })

preds = pd.DataFrame(preds)

0it [00:00, ?it/s]

{'retriever': {'documents': [Document(id=821, content: 'Dodatkowa prowizja od sprzedaży w wyróżnionych ofertach obowiązuje w kategoriach objętych standardow...', score: 45.881447), Document(id=685, content: 'Naliczamy każdorazowo w chwili sprzedaży, podobnie jak w przypadku standardowej prowizji od sprzedaż...', score: 30.520634), Document(id=6, content: 'Dodatkową prowizję od sprzedaży w wyróżnionych ofertach naliczamy we wszystkich ofertach z aktywną o...', score: 25.895947), Document(id=218, content: 'Dodatkową prowizję zwrócimy Ci na takich samych zasadach, jak standardową prowizję od sprzedaży.', score: 20.520496), Document(id=379, content: 'Tak, dodatkowa prowizja obowiązuje konta zwykłe oraz konta Firma.', score: 18.640553), Document(id=509, content: 'Wysokość prowizji od sprzedaży nie jest jednakowa dla wszystkich ofert. Zależy od
* ceny końcowej, z...', score: 17.054335), Document(id=388, content: 'Pobieramy następujące opłaty:
* za wystawienie przedmiotu,
* prowizję od sprzed

In [45]:
preds.to_csv(os.path.join(dataset_path, 'submission.tsv'), sep='\t', index=False)